In [16]:
import tweepy
import time
import json
import speech_recognition as sr
recognizer = sr.Recognizer()
import pyttsx3 

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAFqgyQEAAAAAN6aY80mL3aFMJ2UoE%2Fk3%2FpdSh%2FU%3DmL6jHpKjuBFHXH1eDzZHDNgS7zPIbNRjApewkCEvOw7mznt8rg'
#bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKgSyQEAAAAAdO5iRe0vzyOHTQ9u%2B8jx66yQvEM%3DXlNRezskI4JXeHF5eQxx1XCUMBg9I2JGNvcxpuVfP95l5wlE1t'
consumer_key = 'RlyjyYEh8D4XMfOnq7sHH53lF'
consumer_secret = 'Vh1lcDpIDrBx07lBMFs8NyleDFK81Gi4Q7cP9RU4wdbRDI2tEP'
access_token = '1877804744132931584-kbdfA3jPOnY1HbrhC4sbUnvDLxY57S'
access_token_secret = 'kkv3aLromsqNnmLCdni41bWrAQhxZ6jCNQlkhQu84pfAn'
# Crear el cliente de la API
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret
)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

client = tweepy.Client(bearer_token=bearer_token)

In [17]:
import requests
from requests_oauthlib import OAuth1Session

url = "https://api.x.com/2/users/search"

headers = {"Authorization": f"Bearer {bearer_token}"}
params = {"query": "Coca-Cola", "user.fields": ["name", "username", "verified"]}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    for user in data["data"]:
        if user["verified"]:  # Check for verified account (optional)
            print(f"Nombre: {user['name']}")
            print(f"Usuario: {user['username']}")
            print(f"Verificada: {user['verified']}")
            break  # Assuming you only want the first verified account
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Error: 403
{
  "title": "Unsupported Authentication",
  "detail": "Authenticating with OAuth 2.0 Application-Only is forbidden for this endpoint.  Supported authentication types are [OAuth 1.0a User Context, OAuth 2.0 User Context].",
  "type": "https://api.twitter.com/2/problems/unsupported-authentication",
  "status": 403
}


In [18]:
#Funciona
def obtener_usernames_por_texto(client, texto, max_results=99, wait_time=30):
    usernames = []
    try:
        # Busca tweets recientes que contengan el texto
        response = client.search_recent_tweets(
            query=texto,
            tweet_fields=["author_id"],
            expansions=["author_id"],
            max_results=max_results
        )
        
        if "users" in response.includes:
            for user in response.includes["users"]:
                usernames.append(user["username"])
        return usernames
    except tweepy.errors.TooManyRequests:
        print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
        time.sleep(wait_time)
        return obtener_usernames_por_texto(client, texto, max_results, wait_time=wait_time)
    except Exception as e:
        print(f"Error al obtener usernames: {e}")
        return usernames

In [6]:
#Funciona
def cuenta_verificada(username, wait_time = 30):

    try: 
        user = client.get_user(username=username, user_fields = ['id', 'username', 'name', 'created_at', 'location', 'description', 'public_metrics', 'verified'])
        
        user_data = user.data
        
        print(f"Nombre: {user_data.name}")
        print(f"Usuario: {user_data.username}")
        print(f"ID: {user_data.id}")
        print(f"Fecha de Creacion: {user_data.created_at}")
        print(f"Descripcion: {user_data.description}")
        print(f"Locacion: {user_data.location}")
        print(f"Seguidores: {user_data.public_metrics['followers_count']}")
        print(f"Seguidos: {user_data.public_metrics['following_count']}")
        
        if user_data.verified:
            print(f"{user_data.name} es una cuenta verificada")
        else:
            print(f"{user_data.name} NO es una cuenta verificada")
        
        return user_data
        
    except tweepy.errors.TooManyRequests:
        print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
        time.sleep(wait_time)
        return cuenta_verificada(username, wait_time=wait_time)
    except tweepy.errors.NotFound:
        print("La cuenta no existe.")
    except tweepy.TweepyException as e:
        print(f"Error al recuperar información: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

In [7]:
#Funciona
def buscar_tweets_en_cuenta(username, wait_time=30, max_results=10):
    query = f"from:{username}"  # Filtra tweets del usuario especificado
    while True:
        try:
            response = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['created_at', 'text'])
            if response.data:
                print(f"Tweets de @{username}:")
                for tweet in response.data:
                    print(f"\nFecha: {tweet.created_at}")
                    print(f"Tweet: {tweet.text}")
            else:
                print(f"No se encontraron tweets recientes para la cuenta @{username}.")
            break
        except Exception as e:
            if "429" in str(e):
                print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Otro error ocurrió: {e}")
                break


In [23]:
#Con la version 1.1 deberia funcionar este, verificar credenciales
#No funciono tampoco con la version 1.1, esta obsoleto.

from requests_oauthlib import OAuth1Session
def buscar_usuarios(query, wait_time=30):
    try:
        users = api.search_users(query, user_fields=["username", "id"])
        for user in users.data:
            print(user.username)
            
    except tweepy.errors.TooManyRequests:
        print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
        time.sleep(wait_time)
        return cuenta_verificada(username, wait_time=wait_time)
    except tweepy.TweepyException as e:
        print(f"Error al recuperar información: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")


In [11]:
cuenta_verificada('cocacolamx')

Nombre: Coca-Cola México
Usuario: CocaColaMx
ID: 145019819
Fecha de Creacion: 2010-05-17 23:03:04+00:00
Descripcion: Lleva #LaMagiaDeVerdad a la mesa con Coca-Cola Sin Azúcar. 🥤✨❤️
Locacion: México
Seguidores: 735281
Seguidos: 33
Coca-Cola México es una cuenta verificada


<User id=145019819 name=Coca-Cola México username=CocaColaMx>

In [24]:
buscar_usuarios('cocacola')

Unexpected parameter: user_fields


Error al recuperar información: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product


In [10]:
buscar_tweets_en_cuenta('cocacolamx')

NameError: name 'buscar_tweets_en_cuenta' is not defined

In [20]:
obtener_usernames_por_texto(client, texto = "#cocacola", max_results=99)

['federicoramos40',
 'Freegart01',
 'MCdaRealKang',
 'DadeOyuncak',
 'JoeNam11',
 'NAO_KN_41',
 'CozyByKathy',
 'PrimeRx_io',
 'GrandeConsumo',
 'favour96043',
 'awrk_skibidi',
 'DiningCooking',
 'freakpost11',
 'JRLandaverde',
 'LoriPlanken',
 'iranradioid',
 'KnotSimple',
 'fuzuli41',
 'ElTutito2',
 'BooBooSparkleU1',
 'MuseumANothing',
 'nimzakalz',
 'Ethic_',
 'MX1000n',
 'OconnellAr70870',
 'e4mtweets',
 'ProfttsTaken',
 'kanhomkeng',
 'lindas_craft',
 'EthicLab',
 'VMagufuli',
 'CoachNickMoney',
 'smartKPIs',
 'Val_nTech',
 'COMESA_HQ',
 'esloganmagazine',
 'kznathletics',
 'GagitaE6',
 'impresosrgd',
 'atta_wimon',
 'CatchUp_asia',
 'UbiquitousTaxis',
 'ntrai86',
 'TheLayoffBot',
 'fortuneturkiye',
 'Emixam20',
 'bukunmi_george',
 'NTrowong',
 'Witschy',
 'Virginie_441',
 'Jennife83112653',
 'finans_anlik',
 'ODedOnRealityTV',
 'Mony_Mo02',
 'Pear_sweetmoon',
 'SantyBkpp',
 'lordung',
 'Gibraltar_Group',
 'LesaviezvousQt',
 'akkianvikas',
 'EyKallmeJb',
 'mymini77',
 'KajalSh175

In [29]:
def main():
    
    texto_a_buscar = "Coca-Cola"
    
    usernames = obtener_usernames_por_texto(client, texto_a_buscar, max_results=99)
    
    print("\nUsernames encontrados:")
    for username in usernames:
        print(username)
    
    print("\nVerificando si las cuentas son verificadas:")
    cuentas_verificadas = []
    
    for username in usernames:
        print(f"\nInformación de la cuenta @{username}:")
        user_data = cuenta_verificada(username)
        if user_data and user_data.verified: 
            cuentas_verificadas.append(username)
    
    print("\nCuentas verificadas encontradas:")
    for username in cuentas_verificadas:
        print(f"@{username}")
    
    print("\nRecuperando tweets de cuentas verificadas:")
    for username in cuentas_verificadas:
        buscar_tweets_en_cuenta(username)

In [ ]:
if __name__ == "__main__":
    main()

Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitudes. Esperando 30 segundos...
Demasiadas solicitud

In [27]:
def buscar_cuentas_relacionadas(client, posibles_nombres):
    usernames = []
    for nombre in posibles_nombres:
        try:
            user = client.get_user(username=nombre)
            usernames.append(user.data.username)
        except Exception as e:
            if "429" in str(e):
                print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Otro error ocurrió: {e}")
                break
        return usernames

posibles_nombres = ["CocaCola", "CocaColaMx", "Coke", "Coca-Cola", "Coca", "Cola"]
cuentas_relacionadas = buscar_cuentas_relacionadas(client, posibles_nombres)
print(cuentas_relacionadas)


['CocaCola']


In [2]:
#Prueba Selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def google_search_links(query, max_results=10):
    # Configurar Selenium con el controlador de Chrome
    driver = webdriver.Chrome()  # Cambiar a otro controlador si usas otro navegador
    driver.get("https://www.google.com")

    # Encontrar el cuadro de búsqueda e ingresar la consulta
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)  # Simular presionar "Enter"
    
    # Esperar a que se carguen los resultados
    time.sleep(20)
    
    # Encontrar los enlaces de los resultados
    search_results = driver.find_elements(By.XPATH, "//div[@class='tF2Cxc']/div/a")  # Clase específica para resultados
    links = [result.get_attribute("href") for result in search_results]

    # Limitar los resultados si se solicita
    links = links[:max_results]

    # Cerrar el navegador
    driver.quit()
    
    return links

# Llamar a la función
query = "cocacola"
links = google_search_links(query)
for i, link in enumerate(links, start=1):
    print(f"{i}. {link}")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF71ADC80D5+2992373]
	(No symbol) [0x00007FF71AA5BFD0]
	(No symbol) [0x00007FF71A8F590A]
	(No symbol) [0x00007FF71A8CF4F5]
	(No symbol) [0x00007FF71A9763A7]
	(No symbol) [0x00007FF71A98EE72]
	(No symbol) [0x00007FF71A96F113]
	(No symbol) [0x00007FF71A93A918]
	(No symbol) [0x00007FF71A93BA81]
	GetHandleVerifier [0x00007FF71AE26A2D+3379789]
	GetHandleVerifier [0x00007FF71AE3C32D+3468109]
	GetHandleVerifier [0x00007FF71AE30043+3418211]
	GetHandleVerifier [0x00007FF71ABBC78B+847787]
	(No symbol) [0x00007FF71AA6757F]
	(No symbol) [0x00007FF71AA62FC4]
	(No symbol) [0x00007FF71AA6315D]
	(No symbol) [0x00007FF71AA52979]
	BaseThreadInitThunk [0x00007FF8BE37259D+29]
	RtlUserThreadStart [0x00007FF8BF02AF38+40]


In [ ]:
#Search de google
def search(query):
    try: 
        from googlesearch import search
    except:
        pass
    
    urls = []
    for j in search(query, tld="com", num=10, stop=10, pause=2):
        urls.append(j)
    url = str(urls[0])

search('cocacola')

In [26]:
#Funciona
def obtener_usernames_y_menciones(client, texto, max_results=50, wait_time=30):
    usernames = set()
    try:
        # Busca tweets recientes que contengan el texto
        response = client.search_recent_tweets(
            query=texto,
            tweet_fields=["author_id"],
            expansions=["author_id"],
            max_results=max_results
        )
        
        if "users" in response.includes:
            for user in response.includes["users"]:
                usernames.add(user["username"].lower())
        
        if "data" in response:
            for tweet in response.data:
                if "entities" in tweet and "mentions" in tweet["entities"]:
                    for mention in tweet["entities"]["mentions"]:
                        usernames.add(mention["username"].lower())

        return list(usernames)

    except tweepy.errors.TooManyRequests:
        print(f"Demasiadas solicitudes. Esperando {wait_time} segundos...")
        time.sleep(wait_time)
        return obtener_usernames_por_texto(client, texto, max_results, wait_time=wait_time)
    except Exception as e:
        print(f"Error al obtener usernames: {e}")
        return usernames

texto_busqueda = "#CocaCola"
usernames = obtener_usernames_y_menciones(client, texto_busqueda, max_results=10)

print("Lista de usuarios encontrados:")
print(usernames)

Error al obtener usernames: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Lista de usuarios encontrados:
set()
